In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import lightgbm as lgb
import gc
from scipy.stats import pearsonr as p

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

import warnings
warnings.simplefilter(action='ignore', category= FutureWarning)

from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [59]:
df=pd.read_csv("/Users/cgtyk/Desktop/ALPIN/Alpin_dataset.csv")

# Hem verinin orjinal halini tutmak, hemde date değişkenini df verisine çekmek için  versiyon sıfır şekilnde dataframe atadım.

ver0_df=pd.read_csv("/Users/cgtyk/Desktop/ALPIN/Alpin_dataset.csv")

In [ ]:
# Tip değişimi ve zamana yönelik yeni değişkenler


df["Date"] = pd.to_datetime(df["Date"])

df["YarnCode"] = df["YarnCode"].astype("object")
df["Operator"] = df["Operator"].astype("object")
df["Shift"] = df["Shift"].astype("object")
df['Yıl'] = df["Date"].dt.year
df['Ay'] = df["Date"].dt.month
df['Haftanın_Günü'] = df["Date"].dt.weekday
haftanin_gunleri = {0: 'Pazartesi', 1: 'Salı', 2: 'Çarşamba', 3: 'Perşembe', 4: 'Cuma', 5: 'Cumartesi', 6: 'Pazar'}
df['Haftanın_Günü'] = df["Date"].dt.weekday.map(haftanin_gunleri)
df['Vardiya_Saati'] = df['Shift'].map({1: 'Sabah', 2: 'Akşam', 3: 'Gece'})
df.drop("Shift",axis=1,inplace=True)

df.set_index("Date",inplace=True)

In [62]:
# Memory kullanımını azaltma ( kullanmadım)  hafıza sorununu çözen fonksiyonlar


def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
cat_cols=["Vardiya_Saati","Haftanın_Günü","Yıl","Platin","Ability"]
for col in df.columns:
    print(col, df[col].nunique())

In [65]:
# Verideki efficiencyden başka tek sayısal değişken olan 
#   cycletime değişkeninin üzerinden değişken üretme


df['CycleTime_max'] = df.groupby(['MachCode','ProductCode'])['CycleTime'].transform('max')
df['CycleTime_min'] = df.groupby(['MachCode','ProductCode'])['CycleTime'].transform('min')
df['CycleTime_std'] = df.groupby(['MachCode','ProductCode'])['CycleTime'].transform('std')
df['CycleTime_mean'] = df.groupby(['MachCode','ProductCode'])['CycleTime'].transform('mean')


#Cycletime normalizasyonu bir nevi ölçeklendirme

df['CycleTime_max_norm'] = df['CycleTime']/df['CycleTime_max']


#momentum ekleme finansal verilerdeki  geçmiş değerleri  kullanma gibi 

df['CycleTime_momentum'] = df['CycleTime']/df.groupby(['MachCode','ProductCode'])['CycleTime'].transform(lambda x: x.shift(1))
df['CycleTime_momentum_ay'] = df['CycleTime']/df.groupby(['MachCode','ProductCode','Ay'])['CycleTime'].transform('mean')
df['CycleTime_momentum_yıl'] = df['CycleTime']/df.groupby(['MachCode','ProductCode','Yıl'])['CycleTime'].transform('mean')

del df['Ay'], df['Yıl']

In [ ]:
## Rolling değişkenleri

for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    df['rolling_mean_'+str(i)] = df.groupby(['MachCode'])["Efficiency"].transform(lambda x: x.shift(30).rolling(i).mean()).astype(np.float16)
    df['rolling_std_'+str(i)]  = df.groupby(['MachCode'])["Efficiency"].transform(lambda x: x.shift(30).rolling(i).std()).astype(np.float16)
    
    
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        df[col_name] = df.groupby(['MachCode'])["Efficiency"].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)

In [ ]:
# Veri setindeki belirli gruplandırmalara göre Efficiencyy sütununun ortalaması ve standart sapmasını yeni sütunlar olarak ekledim

icols =  [
            ['MachCode'],
            ['ProductCode'],
            ['Ability'],
            ['Platin'],
            ['MachCode', 'ProductCode'],
            ['MachCode', 'Ability'],
            ['ProductCode', 'Platin'],
            ['ProductCode', 'Ability'],
            ['Platin', 'ProductCode'],
            ['Platin', 'ProductCode']
            ]

for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    df['enc'+col_name+'mean'] = df.groupby(col)['Efficiency'].transform('mean').astype(np.float16)
    df['enc'+col_name+'std'] = df.groupby(col)['Efficiency'].transform('std').astype(np.float16)

In [68]:
# Bizim için verideki, 
#  makine verimliliğine etki eden 
#  en önemli iki kategorik değişkeni one hot encoder'a soktum.

one_hot_encoded = pd.get_dummies(df["Ability"], prefix=col)
df = pd.concat([df, one_hot_encoded], axis=1)
df = df.drop("Ability", axis=1) 

one_hot_encoded = pd.get_dummies(df["Platin"], prefix=col)
df = pd.concat([df, one_hot_encoded], axis=1)
df = df.drop("Platin", axis=1) 

In [70]:
df.drop(["GroupCode","Operator","ProductCode","YarnCode","Item","adet","NumuneKod"],axis=1,inplace=True)


In [ ]:
# Veride çok fazla satır olduğundan memory hatası aldım. 
#Ben de 1000 satır 1000 satır şeklinde, 
# True ve False değerlerini 1 ve  0'a dönüştürdüm

chunk_size = 1000

# Sadece bool türündeki sütunları seç
bool_columns = df.select_dtypes(include=bool).columns

for col in bool_columns:
    for start in range(0, len(df), chunk_size):
        end = start + chunk_size
        df.loc[start:end, col] = df.loc[start:end, col].astype(int)

In [ ]:
# Yine aynı şekilde 
#   verideki date değişkeninden elde ettiğim 
#   2 temel kategorik zaman değişkenini  one hot encoder'a soktum

one_hot_encoded = pd.get_dummies(df["Haftanın_Günü"], prefix=col)
df = pd.concat([df, one_hot_encoded], axis=1)
df = df.drop("Haftanın_Günü", axis=1)

one_hot_encoded = pd.get_dummies(df["Vardiya_Saati"], prefix=col)
df = pd.concat([df, one_hot_encoded], axis=1)
df = df.drop("Vardiya_Saati", axis=1)

In [ ]:
# Herbir MachCode için ayrı bir data frame oluşturup, 
#   bu veilere göre her bir machcode için ayrı model kurmak içi aşağıdaki döngüyü yazdım.
# Aslında GroupKfold ile de bu işlemi yapabilirdim. 
# Groupkfold daha prati olduğu için ona geçebilirim.

machcodes_dataframes = []
machcodes_names = []
unique_machcodes = df['MachCode'].unique()

for i, machcode in enumerate(unique_machcodes, start=1):
    machcode_df = df[df['MachCode'] == machcode].copy()
    machcode_name = f'mach_{i}'
    machcodes_dataframes.append(machcode_df)
    machcodes_names.append(machcode_name)

In [ ]:
machcodes_names